In [3]:
import ccxt 
import os
from dotenv import load_dotenv
import pprint
import pandas as pd
import numpy as np

In [4]:
load_dotenv()
def load_bingx():
    
    api_key = os.getenv("api_key")
    api_secret = os.getenv("api_secret")
    bingx = ccxt.bingx({
    'apiKey': api_key,
    'secret': api_secret,
    'options': {
        'defaultType': 'spot'}  # MUY IMPORTANTE para usar futuros
    
    })
    return bingx
bingx = load_bingx()
print(bingx.fetch_balance())

{'info': {'code': '0', 'msg': '', 'debugMsg': '', 'retryable': '0', 'data': {'balances': [{'asset': 'BNB', 'disPlayName': 'BNB', 'free': '0.0005', 'locked': '0'}, {'asset': 'ETH', 'disPlayName': 'ETH', 'free': '0.000073575', 'locked': '0'}, {'asset': 'BTC', 'disPlayName': 'BTC', 'free': '1.23190299e-8', 'locked': '0'}, {'asset': 'USDT', 'disPlayName': 'USDT', 'free': '12.091759196394428', 'locked': '0'}, {'asset': 'VST', 'disPlayName': 'VST', 'free': '166240.09555901', 'locked': '0'}, {'asset': 'ARB', 'disPlayName': 'ARB', 'free': '0', 'locked': '0'}, {'asset': 'NEO', 'disPlayName': 'NEO', 'free': '0', 'locked': '0'}, {'asset': 'USDC', 'disPlayName': 'USDC', 'free': '1.501', 'locked': '2.2'}, {'asset': 'LTC', 'disPlayName': 'LTC', 'free': '0', 'locked': '0'}, {'asset': 'DOT', 'disPlayName': 'DOT', 'free': '0', 'locked': '0'}]}}, 'BNB': {'free': 0.0005, 'used': 0.0, 'total': 0.0005}, 'ETH': {'free': 7.3575e-05, 'used': 0.0, 'total': 7.3575e-05}, 'BTC': {'free': 1.23190299e-08, 'used': 0

In [50]:
pprint.pprint(bingx.fetch_ticker("USDC/USDT"))

{'ask': 0.9994,
 'askVolume': 38245914.652,
 'average': 0.9993,
 'baseVolume': 43286016.37,
 'bid': 0.9993,
 'bidVolume': 15003.434,
 'change': 0.0001,
 'close': 0.9994,
 'datetime': '2025-07-18T23:15:08.633Z',
 'high': 0.9996,
 'indexPrice': None,
 'info': {'askPrice': '0.9994',
          'askQty': '38245914.652',
          'bidPrice': '0.9993',
          'bidQty': '15003.434',
          'closeTime': '1752880508633',
          'highPrice': '0.9996',
          'lastPrice': '0.9994',
          'lowPrice': '0.9990',
          'openPrice': '0.9993',
          'openTime': '1752794108633',
          'priceChange': '0.0001',
          'priceChangePercent': '0.01%',
          'quoteVolume': '43254968.79',
          'symbol': 'USDC-USDT',
          'volume': '43286016.37'},
 'last': 0.9994,
 'low': 0.999,
 'markPrice': None,
 'open': 0.9993,
 'percentage': 0.01,
 'previousClose': None,
 'quoteVolume': 43254968.79,
 'symbol': 'USDC/USDT',
 'timestamp': 1752880508633,
 'vwap': 0.9992827341806044

In [7]:
data = bingx.fetch_ohlcv("USDC/USDT","1d",limit=1)
        #print(simple_arbitrage_usdc_usdt(data))
data= pd.DataFrame(data)
data.rename(columns={0: "timestamp", 1: "open", 2: "high", 3: "low", 4: "close", 5: "volume"}, inplace=True)
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')

In [8]:
data

,timestamp,open,high,low,close,volume
0,2025-07-22 16:00:00,0.9993,0.9995,0.9993,0.9994,7848399.821


In [9]:
def FIBONACCI_PIVOTS(high, low, close):
    """
    Calcula pivotes de Fibonacci y niveles de soporte/resistencia.
    Devuelve: pivot, r1, r2, r3, s1, s2, s3, r0618, r1000, r1618, s0618, s1000, s1618 (todos np.arrays)
    """
    pivot = (high + low + close) / 3
    diff = high - low

    r1 = pivot + (diff * 0.382)
    r2 = pivot + (diff * 0.618)
    r3 = pivot + (diff * 1.000)
    r1618 = pivot + (diff * 1.618)
    s1 = pivot - (diff * 0.382)
    s2 = pivot - (diff * 0.618)
    s3 = pivot - (diff * 1.000)
    s1618 = pivot - (diff * 1.618)

    # Niveles intermedios (opcional)
    r0618 = pivot + (diff * 0.236)
    r1000 = pivot + (diff * 1.000)
    s0618 = pivot - (diff * 0.236)
    s1000 = pivot - (diff * 1.000)

    return pivot, r1, r2, r3, s1, s2, s3

In [12]:
pivot, r1, r2, r3, s1, s2, s3 =FIBONACCI_PIVOTS(data['high'], data['low'], data['close'])
print("Pivot:", pivot)
print("R1:", r1)    
print("R2:", r2)
print("R3:", r3)
print("S1:", s1)
print("S2:", s2)
print("S3:", s3)

Pivot: 0    0.9994
dtype: float64
R1: 0    0.999476
dtype: float64
R2: 0    0.999524
dtype: float64
R3: 0    0.9996
dtype: float64
S1: 0    0.999324
dtype: float64
S2: 0    0.999276
dtype: float64
S3: 0    0.9992
dtype: float64


In [49]:
bingx.create_order(
    symbol='USDC/USDT',
    type='limit',
    side='buy',
    amount=15,
    price=0.95)

{'info': {'symbol': 'USDC-USDT',
  'orderId': '1946347186262278144',
  'transactTime': '1752880305243',
  'price': '0.95',
  'stopPrice': '0',
  'origQty': '15',
  'executedQty': '0',
  'cummulativeQuoteQty': '0',
  'status': 'PENDING',
  'type': 'LIMIT',
  'side': 'BUY',
  'clientOrderID': '',
  'clientUserID': '',
  'msg': '',
  'bxUid': ''},
 'id': '1946347186262278144',
 'clientOrderId': None,
 'symbol': 'USDC/USDT',
 'timestamp': 1752880305243,
 'datetime': '2025-07-18T23:11:45.243Z',
 'lastTradeTimestamp': None,
 'lastUpdateTimestamp': None,
 'type': 'limit',
 'timeInForce': None,
 'postOnly': None,
 'side': 'buy',
 'price': 0.95,
 'triggerPrice': None,
 'stopLossPrice': None,
 'takeProfitPrice': None,
 'average': None,
 'cost': 0.0,
 'amount': 15.0,
 'filled': 0.0,
 'remaining': 15.0,
 'status': 'open',
 'fee': {'currency': 'USDC', 'cost': None},
 'trades': [],
 'reduceOnly': None,
 'fees': [{'currency': 'USDC', 'cost': None}],
 'stopPrice': None}

In [89]:
bingx.fetch_balance()["USDT"]

{'free': 1.5376685963944254, 'used': 14.25, 'total': 15.787668596394425}

In [3]:
c= bingx.fetch_trades("USDC/USDT", limit=2)

In [4]:
c

[{'id': '138833551',
  'info': {'id': '138833551',
   'price': '0.9996',
   'qty': '65.237',
   'time': '1752948256000',
   'buyerMaker': False},
  'timestamp': 1752948256000,
  'datetime': '2025-07-19T18:04:16.000Z',
  'symbol': 'USDC/USDT',
  'order': None,
  'type': None,
  'side': 'buy',
  'takerOrMaker': 'taker',
  'price': 0.9996,
  'amount': 65.237,
  'cost': 65.2109052,
  'fee': {'cost': None, 'currency': None},
  'fees': []},
 {'id': '138833552',
  'info': {'id': '138833552',
   'price': '0.9996',
   'qty': '21.296',
   'time': '1752948256005',
   'buyerMaker': False},
  'timestamp': 1752948256005,
  'datetime': '2025-07-19T18:04:16.005Z',
  'symbol': 'USDC/USDT',
  'order': None,
  'type': None,
  'side': 'buy',
  'takerOrMaker': 'taker',
  'price': 0.9996,
  'amount': 21.296,
  'cost': 21.2874816,
  'fee': {'cost': None, 'currency': None},
  'fees': []}]

In [5]:
bingx.fetch_orders("USDC/USDT")

NotSupported: bingx fetchOrders() is only supported for swap markets

In [6]:
# Para órdenes abiertas en spot
open_orders = bingx.fetch_open_orders("USDC/USDT")
print(open_orders)

# Para órdenes cerradas en spot
closed_orders = bingx.fetch_closed_orders("USDC/USDT")
print(closed_orders)

[]
[{'info': {'symbol': 'USDC-USDT', 'orderId': '1946388702229856256', 'price': '0.9993', 'StopPrice': '0', 'origQty': '2', 'executedQty': '2', 'cummulativeQuoteQty': '1.9986', 'status': 'FILLED', 'type': 'LIMIT', 'side': 'BUY', 'time': '1752890203506', 'updateTime': '1752894608768', 'origQuoteOrderQty': '1.9986', 'clientOrderID': '', 'fee': '0', 'avgPrice': '0.9993', 'clientUserID': ''}, 'id': '1946388702229856256', 'clientOrderId': None, 'symbol': 'USDC/USDT', 'timestamp': 1752890203506, 'datetime': '2025-07-19T01:56:43.506Z', 'lastTradeTimestamp': 1752894608768, 'lastUpdateTimestamp': 1752894608768, 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 0.9993, 'triggerPrice': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'average': 0.9993, 'cost': 1.9986, 'amount': 2.0, 'filled': 2.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'currency': 'USDC', 'cost': '0'}, 'trades': [], 'reduceOnly': None, 'fees': [{'currency': 'USDC', 'cost': 0.0}], 'stopPri

In [8]:
open_orders = bingx.fetch_open_orders("USDC/USDT")
pprint.pprint(open_orders)

# Para órdenes cerradas en spot
closed_orders = bingx.fetch_closed_orders("USDC/USDT")
pprint.pprint(closed_orders)

[]
[{'amount': 2.0,
  'average': 0.9993,
  'clientOrderId': None,
  'cost': 1.9986,
  'datetime': '2025-07-19T01:56:43.506Z',
  'fee': {'cost': '0', 'currency': 'USDC'},
  'fees': [{'cost': 0.0, 'currency': 'USDC'}],
  'filled': 2.0,
  'id': '1946388702229856256',
  'info': {'StopPrice': '0',
           'avgPrice': '0.9993',
           'clientOrderID': '',
           'clientUserID': '',
           'cummulativeQuoteQty': '1.9986',
           'executedQty': '2',
           'fee': '0',
           'orderId': '1946388702229856256',
           'origQty': '2',
           'origQuoteOrderQty': '1.9986',
           'price': '0.9993',
           'side': 'BUY',
           'status': 'FILLED',
           'symbol': 'USDC-USDT',
           'time': '1752890203506',
           'type': 'LIMIT',
           'updateTime': '1752894608768'},
  'lastTradeTimestamp': 1752894608768,
  'lastUpdateTimestamp': 1752894608768,
  'postOnly': None,
  'price': 0.9993,
  'reduceOnly': None,
  'remaining': 0.0,
  'side': 